# NEW STYLE


In [ ]:
def lnk_to_param(param, link_text=None):
    if link_text is None:
        link_text = f"`{param}`"
    return f"[{link_text}](<project:#{param}>)"


dict = {}

dict["whichInitEquilibrium"] = {
    "mandatory": True,
    "description": [
        "How initial guess is computed.",
        "Either from boundary and axis parameters, or from VMEC file.",
    ],
    # "required_if": "",
    # "default":"",
    "type": "`integer`",
    # "allowed": '',
    "allowed_table": [
        ("`0`", "from axis and boundary parameters"),
        (
            "`1`",
            "from VMEC file, **needs:** "
            + lnk_to_param("VMECwoutfile")
            + ","
            + lnk_to_param("VMECwoutfile_format"),
        ),
    ],
}
dict["VMECwoutfile"] = {
    # "mandatory": True
    "description": [
        "full file name of vmec solution file, either as netcdf or as nemec output,",
        "see " + lnk_to_param("VMECwoutfile_format"),
    ],
    "required_if": lnk_to_param(
        "whichInitEquilibrium", link_text="`whichInitEquilibrium=1`"
    ),
    "type": "`string`",
    # "allowed": '',
    # "default":"",
}

In [ ]:
def generate_doc_param(param, vals):
    outstr = []
    #:::{dropdown} `whichInitEquilibrium` (mandatory)

    outstr.append(f":::{{dropdown}} `{param}`")
    if vals.get("mandatory"):
        outstr[-1] += " (mandatory)"
    outstr.append(f":name: {param}")
    outstr.append("")
    outstr += vals["description"]
    outstr.append("")
    if vals.get("mandatory"):
        outstr.append("**Required Input**")
        outstr.append("")
    if vals.get("required_if"):
        outstr.append(f"**Required if:** {vals['required_if']}")
        outstr.append("")

    outstr.append(f"**Type:** {vals['type']}")
    outstr.append("")

    if vals.get("allowed"):
        outstr.append(f"**Allowed Values:** {vals['allowed']}")
        outstr.append("")

    if table := vals.get("allowed_table"):
        outstr.append("**Allowed Values:**")
        if any("\*" in row[0] for row in table):
            outstr[-1] += " (\* default)"
        outstr.append(":::{list-table}")
        for row in table:
            outstr.append(f"*   - {row[0]}")
            outstr.append(f"    - {row[1]}")
        outstr.append(":::")  # list-table
        outstr.append("")

    if vals.get("default"):
        outstr.append(f"**Default:** {vals['default']}")
        outstr.append("")

    outstr.append(":::")  # dropdown
    return outstr

In [ ]:
out = []
for param, vals in dict.items():
    out += generate_doc_param(param, vals)

with open("../user/minimizer-parameters.md", "w") as f:
    for line in out:
        f.write(line + "\n")

In [ ]:
# this has to fail, prevents the next cells to be executed
assert False

# FIRST ATTEMPT

In [ ]:
# functions to write the dropdown + card in markdown


def block(nlevel, command, title, description, opt=None):
    """
    writes the description in a markdown block, with a header with leading ":"nlevel  and a footer with ":"*nlevel. the command and title are in the header, the option `opt` is possible to add after the header, then the description is added (must be a list of strings. Output is a list of strings for each line.
    """
    outstr = []
    outstr.append(":" * nlevel + "{" + command + "} " + title)
    if opt:
        outstr.append(opt)
    # outstr.append('<'*nlevel) #debug
    outstr.append("")
    [outstr.append(x) for x in description]
    # outstr.append('>'*nlevel)#debug
    outstr.append("")
    outstr.append(":" * nlevel)
    return outstr


def generate_doc_param(param, vals):
    """
    generate the markdown documentation for one parameter, using a dropdown with cards inside.
    """
    descr = []
    [descr.append(x) for x in vals["Description"]]
    descr.append("")
    descr.append("**Options:** (*\*default*)")
    descr.append("| | |")
    descr.append("| :-- | :-- |")
    for opt, exp in vals["Options"].items():
        descr.append(f"| `{opt}` | {exp} |")

    top_content = block(5, "grid-item-card", f"`{param}`", descr)

    top_block = block(6, "grid", "1 1 1 1", top_content)

    col_left_a = block(3, "grid-item-card", "Dependency", vals["Dependency"])
    col_left_block = block(4, "grid", "1 1 1 1", col_left_a, opt=":gutter: 1")
    col_left = block(5, "grid-item", "", col_left_block)

    col_right_a = block(3, "grid-item-card", "Proposal", vals["Proposal"])

    typ = [f'`{vals["Datatype"]}`, {vals["Size"]}']
    col_right_b = block(3, "grid-item-card", "Datatype", typ)
    col_right_content = col_right_a
    col_right_content.append("")
    [col_right_content.append(x) for x in col_right_b]
    col_right_block = block(4, "grid", "1 1 1 1", col_right_content, opt=":gutter: 1")
    col_right = block(5, "grid-item", "", col_right_block)
    two_cols_content = col_left
    two_cols_content.append("")
    [two_cols_content.append(x) for x in col_right]
    two_cols = block(6, "grid", "1 1 2 2", two_cols_content, opt=":gutter: 1")

    allblocks = top_block
    allblocks.append("")
    [allblocks.append(x) for x in two_cols]

    return block(7, "dropdown", f"`{param}`", allblocks)

In [ ]:
dict = {}

dict["MinimizerType"] = {
    "Description": [r"so it is, $a/b=c$ and so it goes on."],
    "Options": {"0": r"gradient descent in $X^1$", "10": r"accel. in $X^2$"},
    "Datatype": "integer",
    "Size": "scalar",
    "Dependency": [
        r"* is only active if $x_2=x_1$ ",
        r"* has nothing else",
        r"* or one more thing $k=1$",
    ],
    "Proposal": [r" if not provided, set `=LA_deg` "],
}
dict["MinimizerTol"] = {
    "Description": [r"so it is, $a/b=c$", "", r" and so it goes on."],
    "Options": {"0": r"gradient descent in $X^1$", "10": r"accel. in $X^2$"},
    "Datatype": "integer",
    "Size": "scalar",
    "Dependency": [
        r"* is only active if $x_2=x_1$ ",
        r"* has nothing else",
        r"* or one more thing $k=1$",
    ],
    "Proposal": [r" if not provided, set `=LA_deg` "],
}

In [ ]:
# loop over all parameters in dict
outstring = []
for key, vals in dict.items():
    tmpstr = generate_doc_param(key, vals)
    [outstring.append(x) for x in tmpstr]
    outstring.append("")

with open("../user/minimizer-parameters.md", "w") as f:
    for line in outstring:
        f.write(line + "\n")